# TEXT SUMMARIZATION USING BART TRANSFORMER MODEL

BART - Bidirectional and Auto Regressive Transformers

#### LOADING THE DATASET

In [1]:
!pip install transformers datasets rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=15c2c47150d9ff1eef420b27be337ea2823686519d089dc0045323d6bf1b7c41
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [2]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [3]:
from datasets import load_dataset

# Use a custom cache directory (e.g., in the current working dir)
dataset = load_dataset("multi_news", cache_dir="./custom_cache", download_mode="force_redownload")
print(dataset["train"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

{'document': 'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter. \n \n Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy 

In [4]:
# Extract each split into a variable
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

# Optionally, convert to lists of texts and summaries
train_texts = train_data["document"]
train_summaries = train_data["summary"]

val_texts = val_data["document"]
val_summaries = val_data["summary"]


In [5]:
from transformers import AutoTokenizer

# Use appropriate tokenizer (e.g., for BART or T5)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Tokenize the dataset
def tokenize_fn(example):
    return tokenizer(example["document"], truncation=True, padding="max_length", max_length=1024)

def tokenize_target(example):
    return tokenizer(example["summary"], truncation=True, padding="max_length", max_length=128)

tokenized_train = train_data.map(lambda x: {**tokenize_fn(x), "labels": tokenize_target(x)["input_ids"]}, batched=True)
tokenized_val = val_data.map(lambda x: {**tokenize_fn(x), "labels": tokenize_target(x)["input_ids"]}, batched=True)


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [6]:
!pip install -U transformers


In [7]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM
from transformers import EarlyStoppingCallback

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Evaluation is done at the end of each epoch
    save_strategy="epoch",  # Model is saved at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,  # Set a higher number of epochs here
    predict_with_generate=True,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss", # Early stopping monitors this metric
    greater_is_better=False, # For eval_loss, smaller is better
)

# Define the EarlyStoppingCallback
# patience=10 means it will stop if eval_loss doesn't improve for 10 consecutive evaluation steps (epochs in this case)
# min_delta=0.0 means it will stop if the improvement is less than 0.0
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)


# Define trainer with early stopping
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback] # Add the early stopping callback here
)

# Start training
trainer.train()

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/tmp/ipython-input-7-4144618824.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: armond-jose (armond-jose-vimal-jyothi-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,2.522800,2.312682


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=22486, training_loss=2.6222067200754777, metrics={'train_runtime': 8879.2739, 'train_samples_per_second': 5.065, 'train_steps_per_second': 2.532, 'total_flos': 2.742106799996928e+16, 'train_loss': 2.6222067200754777, 'epoch': 1.0})

In [8]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)

# Save the model and tokenizer after training
model.save_pretrained("/content/your_model_directory")
tokenizer.save_pretrained("/content/your_model_directory")

{'eval_loss': 2.3126821517944336, 'eval_runtime': 280.62, 'eval_samples_per_second': 20.034, 'eval_steps_per_second': 10.017, 'epoch': 1.0}


('/content/your_model_directory/tokenizer_config.json',
 '/content/your_model_directory/special_tokens_map.json',
 '/content/your_model_directory/vocab.json',
 '/content/your_model_directory/merges.txt',
 '/content/your_model_directory/added_tokens.json',
 '/content/your_model_directory/tokenizer.json')

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/your_model_directory")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/your_model_directory")

# Function to summarize text
def summarize(text):
    # Tokenize the input text
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Summarize examples from the test set (e.g., first two examples)
print("Summaries from test set:")
for i in range(2):
    test_text = test_data["document"][i]
    generated_summary = summarize(test_text)
    print(f"Original Text {i+1}: {test_text[:200]}...") # Print a snippet of the original text
    print(f"Generated Summary {i+1}: {generated_summary}")
    print("-" * 50)


# Summarize your own custom text
custom_text = """
This is an example of a custom piece of text that I want to summarize.
It can be a paragraph, an article, or any other long form content.
The model will try to generate a concise summary of this text.
Let's see how well it performs on this custom input.
"""
print("\nSummary of custom text:")
custom_summary = summarize(custom_text)
print("Original Text:", custom_text)
print("Generated Summary:", custom_summary)

Summaries from test set:
Original Text 1: GOP Eyes Gains As Voters In 11 States Pick Governors 
 
 Enlarge this image toggle caption Jim Cole/AP Jim Cole/AP 
 
 Voters in 11 states will pick their governors tonight, and Republicans appear on ...
Generated Summary 1: – Voters in 11 states will pick their governors tonight, and Republicans appear on track to increase their numbers by at least one, with the potential to extend their hold to more than two-thirds of the nation's top state offices. Eight of the gubernatorial seats up for grabs are now held by Democrats; three are in Republican hands. Republicans currently hold 29 governorships, Democrats have 20, and Rhode Island's Gov. Lincoln Chafee is an Independent. Polls and race analysts suggest that only three of tonight's contests are considered competitive, all in states where incumbent Democratic governors aren't running again: Montana, New Hampshire, and
--------------------------------------------------
Original Text 2: 
 
 
 
 U

In [10]:
!zip -r /content/your_model_directory.zip /content/your_model_directory

  adding: content/your_model_directory/ (stored 0%)
  adding: content/your_model_directory/vocab.json (deflated 59%)
  adding: content/your_model_directory/special_tokens_map.json (deflated 52%)
  adding: content/your_model_directory/merges.txt (deflated 53%)
  adding: content/your_model_directory/tokenizer_config.json (deflated 75%)
  adding: content/your_model_directory/generation_config.json (deflated 47%)
  adding: content/your_model_directory/config.json (deflated 64%)
  adding: content/your_model_directory/tokenizer.json (deflated 82%)
  adding: content/your_model_directory/model.safetensors (deflated 7%)


In [21]:
# Your provided text
text_to_summarize = """
Azerbaijan has recently moved to strike a peace deal with Armenia after violently
recapturing the disputed territory of Nagorno-Karabakh in 2023. But continued fighting
 between the two countries might be more likely than peace, and other transnational
  threats continue to lurk beneath the surface..
"""

# Call the summarize function with the text
generated_summary = summarize(text_to_summarize)

# Print the original text and the generated summary
print("Original Text:")
print(text_to_summarize)
print("\nGenerated Summary:")
print(generated_summary)

Original Text:

Azerbaijan has recently moved to strike a peace deal with Armenia after violently 
recapturing the disputed territory of Nagorno-Karabakh in 2023. But continued fighting
 between the two countries might be more likely than peace, and other transnational
  threats continue to lurk beneath the surface..


Generated Summary:
– A ceasefire deal between Azerbaijan and Armenia is in the works, but the two countries are still fighting over the disputed territory of Nagorno-Karabakh in 2023, the New York Times reports. The two countries have agreed to a peace deal, but it's not clear how long it will last, and the Times notes that the two sides have been fighting for years. "We're not going to be able to reach a peace agreement," the Times quotes an unnamed official as saying. "It's too early to say whether it will be a long-term deal."


In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

fine_tuned_model_placeholder = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
fine_tuned_tokenizer_placeholder = AutoTokenizer.from_pretrained("facebook/bart-base")


# Your provided text
text_to_summarize = """
Azerbaijan has recently moved to strike a peace deal with Armenia after violently recapturing the disputed territory of Nagorno-Karabakh in 2023. But continued fighting between the two countries might be more likely than peace, and other transnational threats continue to lurk beneath the surface. For one, Azerbaijan’s President Ilham Aliyev retains visions of territorial expansion through a creeping invasion of Armenia’s southern border. And the tentacles of Azerbaijan’s business dealings should raise alarms throughout the international financial system.

Despite efforts by the Azerbaijani government to portray itself as a respectable member of the international community by hosting the COP29 climate change conference in November 2024 and other international events, the regime in Baku remains highly militarized and does not play by the rules. Since 2020, Azerbaijan’s military has occupied an estimated 83 square miles of Armenian land, resulting in the death of hundreds of soldiers and the displacement of more than 7,600 civilians from Armenian provinces. This does not include the forcible displacement of nearly all of Nagorno-Karabakh’s ethnic Armenian population – more than 100,000 people – just two years ago.

Azerbaijan wants some of the land to build a railroad to its exclave Nakhichevan, in a project that Aliyev has dubbed the “Zangezur corridor.” Armenia opposes Azerbaijan’s plan, saying it would breach Armenia’s sovereignty. This has not been part of the negotiations over a peace agreement, though both sides have agreed to address it separately after a deal is struck.

"""

# Function to summarize text using a given model and tokenizer (using the placeholders)
def summarize_with_placeholder(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Call the summarize function with the placeholder model and tokenizer
generated_summary = summarize_with_placeholder(text_to_summarize, fine_tuned_model_placeholder, fine_tuned_tokenizer_placeholder)


# Print the original text and the generated summary
print("Original Text:")
print(text_to_summarize)
print("\nGenerated Summary (using the 'fine_tuned_model_placeholder'):")
print(generated_summary)

Original Text:

Azerbaijan has recently moved to strike a peace deal with Armenia after violently recapturing the disputed territory of Nagorno-Karabakh in 2023. But continued fighting between the two countries might be more likely than peace, and other transnational threats continue to lurk beneath the surface. For one, Azerbaijan’s President Ilham Aliyev retains visions of territorial expansion through a creeping invasion of Armenia’s southern border. And the tentacles of Azerbaijan’s business dealings should raise alarms throughout the international financial system.

Despite efforts by the Azerbaijani government to portray itself as a respectable member of the international community by hosting the COP29 climate change conference in November 2024 and other international events, the regime in Baku remains highly militarized and does not play by the rules. Since 2020, Azerbaijan’s military has occupied an estimated 83 square miles of Armenian land, resulting in the death of hundreds 